Buka Zip

In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile('/content/MultiClass Image Classification.zip','r') as zipObj:
  zipObj.extractall('MultiClass_Image_Classification')

Import Library yang diperlukan

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import matplotlib.pyplot as plt
from PIL import Image
import os
import tensorflow as tf
import numpy as np


# Analisa sederhana terhadap data

## Cek Jumlah data masing-masing kelas

In [ ]:
def count_files(folder_path):
  count = 0
  for root, _, files in os.walk(folder_path):
    for _ in files:
      count += 1
  return count

folder_path = "/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-lasem"
file_count = count_files(folder_path)
print(f"Jumlah file di folder '{folder_path}': {file_count}")


Jumlah file di folder '/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-lasem': 50


In [ ]:
folder_path = "/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-megamendung"
file_count = count_files(folder_path)
print(f"Jumlah file di folder '{folder_path}': {file_count}")

Jumlah file di folder '/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-megamendung': 47


In [ ]:
folder_path = "/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-parang"
file_count = count_files(folder_path)
print(f"Jumlah file di folder '{folder_path}': {file_count}")

Jumlah file di folder '/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-parang': 50


***Terdapat masing-masing 50 foto pada batik-parang dan batik-lasem terdapat 47 foto pada batik megamendung***

Terdapat potensi overfitting karena jumlah class yang berbeda.

Solusi yang kami pikrikan adalah drop dari masing-masing batik-lasem dan batik- parang.Namun sebelum itu, kami ingin tau, apakah ada foto yang corrupt.

## Cek ada data yang corrupt gak

In [ ]:
!pip install Pillow
import os
from PIL import Image

direktori = "/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-megamendung"
ada_corrupt = False

for filename in os.listdir(direktori):
    filepath = os.path.join(direktori, filename)
    try:

        img = Image.open(filepath)
        img.verify()
    except (IOError, SyntaxError) as e:
        print(f"Foto '{filename}' corrupt: {e}")
        ada_corrupt = True

if not ada_corrupt:
    print("Tidak ada foto corrupt yang ditemukan di batik-megamendung")

Foto '49.jpg' corrupt: cannot identify image file '/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-megamendung/49.jpg'


In [ ]:
direktori = "/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-parang"
ada_corrupt = False

for filename in os.listdir(direktori):
    filepath = os.path.join(direktori, filename)
    try:

        img = Image.open(filepath)
        img.verify()
    except (IOError, SyntaxError) as e:
        print(f"Foto '{filename}' corrupt: {e}")
        ada_corrupt = True

if not ada_corrupt:
    print("Tidak ada foto corrupt yang ditemukan di batik-parang")

Tidak ada foto corrupt yang ditemukan di batik-parang


In [ ]:
direktori = "/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-lasem"
ada_corrupt = False

for filename in os.listdir(direktori):
    filepath = os.path.join(direktori, filename)
    try:

        img = Image.open(filepath)
        img.verify()
    except (IOError, SyntaxError) as e:
        print(f"Foto '{filename}' corrupt: {e}")
        ada_corrupt = True

if not ada_corrupt:
    print("Tidak ada foto corrupt yang ditemukan di batik-lasem")

Tidak ada foto corrupt yang ditemukan di batik-lasem


Ternyata ditemukan 1 foto pada batik-megamendung yang corrupt, sehingga kami memutuskan untuk drop foto tersebut.

## Drop data yang corrupt dan cek apakah sudah kedrop atau belum

In [ ]:
import os
from PIL import Image

def remove_corrupt_images(directory):
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            try:

                img = Image.open(filepath)

                img.verify()
            except (IOError, SyntaxError) as e:
                print(f"Corrupt image found: {filepath}")
                os.remove(filepath)

dataset_dir = "/content/MultiClass_Image_Classification/MultiClass Image Classification"

for class_folder in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_folder)
    if os.path.isdir(class_path):
        remove_corrupt_images(class_path)

Corrupt image found: /content/MultiClass_Image_Classification/MultiClass Image Classification/batik-megamendung/49.jpg


In [ ]:
def count_files(folder_path):

  count = 0
  for root, _, files in os.walk(folder_path):
    for _ in files:
      count += 1
  return count

folder_path = "/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-megamendung"
file_count = count_files(folder_path)
print(f"Jumlah file di folder '{folder_path}': {file_count}")

Jumlah file di folder '/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-megamendung': 46


## drop untuk batik-lasem dan batik-parang


In [ ]:
import os
import shutil

original_folder_path = "/content/MultiClass_Image_Classification/MultiClass Image Classification"
batik_lasem_path = os.path.join(original_folder_path, "batik-lasem")
batik_parang_path = os.path.join(original_folder_path, "batik-parang")


import random

num_images_to_drop = 4


images_to_drop_lasem = random.sample(os.listdir(batik_lasem_path), num_images_to_drop)


images_to_drop_parang = random.sample(os.listdir(batik_parang_path), num_images_to_drop)

for image in images_to_drop_lasem:
  image_path = os.path.join(batik_lasem_path, image)
  os.remove(image_path)

for image in images_to_drop_parang:
  image_path = os.path.join(batik_parang_path, image)
  os.remove(image_path)


In [ ]:
folder_path = "/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-parang"
file_count = count_files(folder_path)
print(f"Jumlah file di folder '{folder_path}': {file_count}")

Jumlah file di folder '/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-parang': 46


In [ ]:
folder_path = "/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-lasem"
file_count = count_files(folder_path)
print(f"Jumlah file di folder '{folder_path}': {file_count}")

Jumlah file di folder '/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-lasem': 46


## Cek size dan resolusi foto

In [ ]:
folder = "/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-megamendung"
foto_terburuk = []
for filename in os.listdir(folder):

    if filename.endswith(('.jpg', '.jpeg', '.png')):

        try:
            image = Image.open(os.path.join(folder, filename))
        except:
            continue

        width, height = image.size


        area = width * height


        foto_terburuk.append((filename, width, height, area))


foto_terburuk.sort(key=lambda x: x[3])

print("5 foto dengan resolusi terburuk di batik-megamendung:")
for i in range(5):
    filename, width, height, area = foto_terburuk[i]
    print(f"{i+1}. {filename}: {width}x{height} ({area}px)")




5 foto dengan resolusi terburuk di batik-megamendung:
1. 3.jpg: 236x337 (79532px)
2. 12.jpg: 400x280 (112000px)
3. 5.jpg: 280x400 (112000px)
4. 26.jpg: 400x300 (120000px)
5. 22.jpg: 400x300 (120000px)


In [ ]:
folder = "/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-lasem"
foto_terburuk = []
for filename in os.listdir(folder):

    if filename.endswith(('.jpg', '.jpeg', '.png')):

        try:
            image = Image.open(os.path.join(folder, filename))
        except:
            continue

        width, height = image.size


        area = width * height


        foto_terburuk.append((filename, width, height, area))


foto_terburuk.sort(key=lambda x: x[3])

print("5 foto dengan resolusi terburuk di batik-lasem:")
for i in range(5):
    filename, width, height, area = foto_terburuk[i]
    print(f"{i+1}. {filename}: {width}x{height} ({area}px)")




5 foto dengan resolusi terburuk di batik-lasem:
1. 37.jpg: 200x113 (22600px)
2. 11.jpg: 300x216 (64800px)
3. 12.jpg: 300x225 (67500px)
4. 4.jpg: 320x212 (67840px)
5. 17.jpg: 361x288 (103968px)


In [ ]:
folder = "/content/MultiClass_Image_Classification/MultiClass Image Classification/batik-parang"
foto_terburuk = []
for filename in os.listdir(folder):

    if filename.endswith(('.jpg', '.jpeg', '.png')):

        try:
            image = Image.open(os.path.join(folder, filename))
        except:
            continue

        width, height = image.size


        area = width * height


        foto_terburuk.append((filename, width, height, area))


foto_terburuk.sort(key=lambda x: x[3])

print("5 foto dengan resolusi terburuk di batik-parang:")
for i in range(5):
    filename, width, height, area = foto_terburuk[i]
    print(f"{i+1}. {filename}: {width}x{height} ({area}px)")




5 foto dengan resolusi terburuk di batik-parang:
1. 50.jpg: 229x220 (50380px)
2. 30.jpg: 305x218 (66490px)
3. 26.jpg: 300x225 (67500px)
4. 5.jpg: 310x220 (68200px)
5. 2.jpg: 300x232 (69600px)


dari hasil tersebut kita bisa tau bahwa ukuran-ukuran foto pada data kita berbeda-beda.

# Data prep & Model

In [ ]:
IMAGE_SIZE=224
BATCH_SIZE=64

In [ ]:
base_dir = "/content/MultiClass_Image_Classification/MultiClass Image Classification"

In [ ]:
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
    )

test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255,
     validation_split=0.1
)

train_datagen=train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
)

test_datagen=test_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 126 images belonging to 3 classes.
Found 12 images belonging to 3 classes.


## Model 1

In [ ]:
cnn=tf.keras.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=64,padding='same',strides=2,kernel_size=3,activation='relu',input_shape=(224,224,3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(3,activation='softmax'))

In [ ]:
from tensorflow.keras.utils import plot_model

print(cnn.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 56, 56, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 32)          9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 3, 3, 32)          0

In [ ]:
cnn.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

cnn.fit(train_datagen, epochs=20, validation_data=test_datagen, callbacks=[early_stopping])

Epoch 1/20
2/2 [==============================] - 7s 3s/step - loss: 0.6710 - accuracy: 0.7698 - val_loss: 0.4810 - val_accuracy: 0.8333
Epoch 2/20
2/2 [==============================] - 5s 2s/step - loss: 0.6685 - accuracy: 0.7619 - val_loss: 0.4535 - val_accuracy: 0.9167
Epoch 3/20
2/2 [==============================] - 6s 3s/step - loss: 0.6282 - accuracy: 0.7460 - val_loss: 0.4262 - val_accuracy: 0.8333
Epoch 4/20
2/2 [==============================] - 5s 2s/step - loss: 0.6372 - accuracy: 0.7937 - val_loss: 0.4220 - val_accuracy: 0.8333
Epoch 5/20
2/2 [==============================] - 7s 3s/step - loss: 0.6280 - accuracy: 0.7857 - val_loss: 0.4238 - val_accuracy: 0.9167
Epoch 6/20
2/2 [==============================] - 5s 2s/step - loss: 0.6005 - accuracy: 0.8175 - val_loss: 0.4057 - val_accuracy: 0.9167
Epoch 7/20
2/2 [==============================] - 6s 3s/step - loss: 0.5772 - accuracy: 0.7937 - val_loss: 0.3847 - val_accuracy: 0.8333
Epoch 7: early stopping


## Model 5

In [ ]:
cnn = tf.keras.Sequential()


cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu', input_shape=(224, 224, 3)))  # Increased filter size, adjusted stride
cnn.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=3, padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())

cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(0.2))

cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=2, padding='same', activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=3, padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(0.4))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(1024, activation='relu'))
cnn.add(tf.keras.layers.Dense(3, activation='softmax'))

In [ ]:
cnn.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

cnn.fit(train_datagen, epochs=20, validation_data=test_datagen, callbacks=[early_stopping])

Epoch 1/20
2/2 [==============================] - 7s 2s/step - loss: 0.8217 - accuracy: 0.7063 - val_loss: 1.0411 - val_accuracy: 0.5000
Epoch 2/20
2/2 [==============================] - 7s 4s/step - loss: 0.5546 - accuracy: 0.8175 - val_loss: 1.0261 - val_accuracy: 0.5000
Epoch 3/20
2/2 [==============================] - 5s 2s/step - loss: 0.3797 - accuracy: 0.8175 - val_loss: 1.0004 - val_accuracy: 0.3333
Epoch 4/20
2/2 [==============================] - 5s 2s/step - loss: 0.5164 - accuracy: 0.7857 - val_loss: 0.9972 - val_accuracy: 0.4167
Epoch 5/20
2/2 [==============================] - 6s 2s/step - loss: 0.4031 - accuracy: 0.8571 - val_loss: 1.0419 - val_accuracy: 0.3333
Epoch 6/20
2/2 [==============================] - 5s 2s/step - loss: 0.3672 - accuracy: 0.8889 - val_loss: 0.9989 - val_accuracy: 0.3333
Epoch 6: early stopping


## Model 6

In [ ]:
cnn = tf.keras.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu', input_shape=(224, 224, 3)))  # Increased filter size, adjusted stride
cnn.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=3, padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())

cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=3, padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(0.2))

cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=2, padding='same', activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=3, padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(0.4))

cnn.add(tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same', activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(0.5))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(1024, activation='relu'))
cnn.add(tf.keras.layers.Dense(3, activation='softmax'))

In [ ]:
cnn.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

cnn.fit(train_datagen, epochs=20, validation_data=test_datagen, callbacks=[early_stopping])

Epoch 1/20
2/2 [==============================] - 9s 2s/step - loss: 0.6901 - accuracy: 0.7698 - val_loss: 1.0694 - val_accuracy: 0.5833
Epoch 2/20
2/2 [==============================] - 6s 2s/step - loss: 0.6565 - accuracy: 0.7460 - val_loss: 1.0676 - val_accuracy: 0.4167
Epoch 3/20
2/2 [==============================] - 6s 2s/step - loss: 0.3884 - accuracy: 0.8333 - val_loss: 1.0617 - val_accuracy: 0.3333
Epoch 4/20
2/2 [==============================] - 5s 2s/step - loss: 0.6189 - accuracy: 0.7857 - val_loss: 1.0559 - val_accuracy: 0.3333
Epoch 5/20
2/2 [==============================] - 7s 3s/step - loss: 0.4687 - accuracy: 0.7937 - val_loss: 1.0493 - val_accuracy: 0.6667
Epoch 6/20
2/2 [==============================] - 5s 2s/step - loss: 0.2687 - accuracy: 0.8968 - val_loss: 1.0549 - val_accuracy: 0.5833
Epoch 7/20
2/2 [==============================] - 6s 3s/step - loss: 0.6182 - accuracy: 0.7937 - val_loss: 1.0702 - val_accuracy: 0.4167
Epoch 8/20
2/2 [=========================

## Model 7


In [ ]:
cnn = tf.keras.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation='relu', input_shape=(224, 224, 3)))  # Increased filter size, adjusted stride
cnn.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=3, padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())


cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=3, padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())


cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=2, padding='same', activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=4, strides=4, padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(0.2))

cnn.add(tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same', activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=4, strides=4, padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(0.5))


cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(1024, activation='relu'))

In [ ]:
cnn.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

cnn.fit(train_datagen, epochs=20, validation_data=test_datagen, callbacks=[early_stopping])

Epoch 1/20
2/2 [==============================] - 8s 3s/step - loss: 1.2766 - accuracy: 0.4603 - val_loss: 1.0849 - val_accuracy: 0.5000
Epoch 2/20
2/2 [==============================] - 5s 2s/step - loss: 0.6644 - accuracy: 0.7778 - val_loss: 1.0803 - val_accuracy: 0.3333
Epoch 3/20
2/2 [==============================] - 4s 2s/step - loss: 0.6874 - accuracy: 0.7302 - val_loss: 1.0752 - val_accuracy: 0.3333
Epoch 4/20
2/2 [==============================] - 6s 3s/step - loss: 0.7533 - accuracy: 0.6825 - val_loss: 1.0680 - val_accuracy: 0.3333
Epoch 5/20
2/2 [==============================] - 4s 2s/step - loss: 0.4691 - accuracy: 0.8175 - val_loss: 1.0621 - val_accuracy: 0.5833
Epoch 6/20
2/2 [==============================] - 5s 2s/step - loss: 0.3775 - accuracy: 0.8254 - val_loss: 1.0574 - val_accuracy: 0.6667
Epoch 7/20
2/2 [==============================] - 6s 2s/step - loss: 0.4568 - accuracy: 0.8254 - val_loss: 1.0508 - val_accuracy: 0.4167
Epoch 8/20
2/2 [=========================